# 1

In [5]:
using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, c >= 0)
@variable(m, p>=0)
@variable(m, s>=0)
@variable(m, a >=14000)
@variable(m, b>=18000)

@objective(m, Min, 0.11*c+0.12*p+0.09*s+0.4(c+p)+0.08*s+0.65*a+0.21*b)

@constraint(m, 5.7*c+8.2*p+6.1*s >= 4.2*(c+s+p))
@constraint(m, 6.1*c+3.4*p+1.8*s >= 3.5*(c+s+p))
@constraint(m, 3.3*c+0.7*p+15*s >= 5.2*(c+s+p))
@constraint(m, c<=12300)
@constraint(m, p<=16000)
@constraint(m, s<=9000)
@constraint(m, c+p+s == a+b)

solve(m)

println("Corn ", getvalue(c))
println("Potato ", getvalue(p))
println("Sugar ",getvalue(s))
println("Powder ",getvalue(a))
println("Paste ",getvalue(b))
println("Min ", getobjectivevalue(m))


Corn 12300.0
Potato 10700.0
Sugar 9000.0
Powder 14000.0
Paste 18000.0
Min 26247.0


# 2


In [108]:
# make lists of products and months in planning horizon 
prods = [:panther, :penguin, :panda, :pangolin] 
months = [:1,:2,:3,:4,:5,:6] 

# make dictionary of demand each month for each product
demand = Dict() 
demand[:panther] = Dict(zip(months,[1350,3000,2500,3500,1500,2000])) 
demand[:panda] = Dict(zip(months,[2500,1700,3000,1800,1300,2000])) 
demand[:penguin] = Dict(zip(months,[1200,600,800,800,1100,900])) 
demand[:pangolin] = Dict(zip(months,[1500,1700,1400,900,1000,1900])) 

# make dictionary mapping each product to its production cost 
cprod = Dict(zip(prods,[22,26,15,19])) 

# make dictionary mapping each product to its inventory cost 
cstore = Dict(zip(prods,[0.5,0.4,0.3,0.3])) 

# make dictionary mapping each product to its starting inventory 
inv_init = Dict(zip(prods,[25,0,5,0]))
# make dictionary mapping each product to its ending inventory 
inv_end = Dict(zip(prods,[100,15,10,10]));

using JuMP, Clp, SCS, ECOS

m = Model(solver=ClpSolver())

@variable(m, X[prods,months]>=0) # production
@variable(m, I[prods,months]>=0) # inventory for each
@variable(m, add[prods,months]>=0) # number of increase in x
@variable(m, de[prods,months]>=0) # number of decrease in x

@objective(m, Min, sum(cprod[i]*X[i,j]+cstore[i]*I[i,j] for i in prods, j in months)+0.75*sum(add)+0.4*sum(de)) 

@constraint(m, all[i in prods], add[i,6] == 0)
@constraint(m, all2[i in prods], de[i,6] == 0)
@constraint(m, increase[i in prods, j in 1:5], X[i,j]+add[i,j]-de[i,j] == X[i,j+1])

@constraint(m, init[i in prods], inv_init[i]+X[i,1] == I[i,1]+demand[i][1])
@constraint(m, iend[i in prods], I[i,6] == inv_end[i])
@constraint(m, production[i in prods, j in 2:6], X[i,j]+I[i,j-1] == demand[i][j] + I[i,j])



solve(m)

println("\$", getvalue(X))
println("total cost: \$", getobjectivevalue(m))


$X: 2 dimensions:
[ panther,:]
  [ panther,1] = 2162.5
  [ panther,2] = 2162.5
  [ panther,3] = 3000.0
  [ panther,4] = 3000.0
  [ panther,5] = 1800.0
  [ panther,6] = 1800.0
[ penguin,:]
  [ penguin,1] = 1200.0
  [ penguin,2] = 735.0
  [ penguin,3] = 735.0
  [ penguin,4] = 915.0
  [ penguin,5] = 915.0
  [ penguin,6] = 915.0
[   panda,:]
  [   panda,1] = 2495.0
  [   panda,2] = 2350.0
  [   panda,3] = 2350.0
  [   panda,4] = 1800.0
  [   panda,5] = 1655.0
  [   panda,6] = 1655.0
[pangolin,:]
  [pangolin,1] = 1600.0
  [pangolin,2] = 1600.0
  [pangolin,3] = 1400.0
  [pangolin,4] = 1270.0
  [pangolin,5] = 1270.0
  [pangolin,6] = 1270.0
total cost: $795073.375


In [110]:
# make lists of products and months in planning horizon 
prods = [:panther, :penguin, :panda, :pangolin] 
months = [:1,:2,:3,:4,:5,:6] 

# make dictionary of demand each month for each product
demand = Dict() 
demand[:panther] = Dict(zip(months,[1350,3000,2500,3500,1500,2000])) 
demand[:panda] = Dict(zip(months,[2500,1700,3000,1800,1300,2000])) 
demand[:penguin] = Dict(zip(months,[1200,600,800,800,1100,900])) 
demand[:pangolin] = Dict(zip(months,[1500,1700,1400,900,1000,1900])) 

# make dictionary mapping each product to its production cost 
cprod = Dict(zip(prods,[22,26,15,19])) 

# make dictionary mapping each product to its inventory cost 
cstore = Dict(zip(prods,[0.5,0.4,0.3,0.3])) 

# make dictionary mapping each product to its starting inventory 
inv_init = Dict(zip(prods,[25,0,5,0]))
# make dictionary mapping each product to its ending inventory 
inv_end = Dict(zip(prods,[100,15,10,10]));

using JuMP, Clp

m = Model(solver=ClpSolver())

@variable(m, X[prods,months]>=0) # production
@variable(m, I[prods,months]) # inventory for each
@variable(m, add[prods,months]>=0) # number of increase in x
@variable(m, de[prods,months]>=0) # number of decrease in x
@variable(m, L[prods,months]>=0) #number of leftover
@variable(m, B[prods,months]>=0)#backlog

@objective(m, Min, sum(cprod[i]*X[i,j]+cstore[i]*L[i,j] for i in prods, j in months)+0.75*sum(add)+0.4*sum(de)+0.45*sum(B)) 

@constraint(m, ab[i in prods], B[i,6] == 0)
@constraint(m, all[i in prods], add[i,6] == 0)
@constraint(m, all2[i in prods], de[i,6] == 0)

@constraint(m, increase[i in prods, j in 1:5], X[i,j]+add[i,j]-de[i,j] == X[i,j+1])

@constraint(m, init[i in prods], inv_init[i]+X[i,1] == I[i,1]+demand[i][1])
@constraint(m, iend[i in prods], I[i,6] == inv_end[i])
@constraint(m, production[i in prods, j in 2:6], X[i,j]+I[i,j-1] == demand[i][j] + I[i,j])

@constraint(m, updown[i in prods, j in months], I[i,j] == L[i,j]-B[i,j])



solve(m)

println("\$", getvalue(X))
println("total cost: \$", getobjectivevalue(m))

$X: 2 dimensions:
[ panther,:]
  [ panther,1] = 2229.166666666666
  [ panther,2] = 2229.166666666666
  [ panther,3] = 2366.6666666666674
  [ panther,4] = 2366.6666666666674
  [ panther,5] = 2366.6666666666665
  [ panther,6] = 2366.6666666666665
[ penguin,:]
  [ penguin,1] = 896.25
  [ penguin,2] = 896.25
  [ penguin,3] = 896.25
  [ penguin,4] = 896.25
  [ penguin,5] = 915.0
  [ penguin,6] = 915.0
[   panda,:]
  [   panda,1] = 2398.333333333333
  [   panda,2] = 2398.333333333333
  [   panda,3] = 2398.333333333334
  [   panda,4] = 1800.0
  [   panda,5] = 1655.0
  [   panda,6] = 1655.0
[pangolin,:]
  [pangolin,1] = 1600.0
  [pangolin,2] = 1600.0
  [pangolin,3] = 1400.0
  [pangolin,4] = 1270.0
  [pangolin,5] = 1270.0
  [pangolin,6] = 1270.0
total cost: $794175.3333333334


# 3

In [34]:
using Plots 
# set x coordinates and y coordinates for each friend 
x_coord = [20,18,0,35,2,11,33,2,4,12,7,26]  
y_coord = [8,13,5,7,11,15,22,7,10,0,6,21] 
# plot locations of friends on a "map" 
plot(x_coord,y_coord,seriestype=:scatter,markershape=:x,    
    markersize=1,legend=false,series_annotations=["1","2","3","4","5","6","7", "8","9","10","11","12",])

friends = [:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12] # list of friends 
# make a list of (x,y) coordinate pairs for each friend 
x_y_coord = [(x_coord[i],y_coord[i]) for i in friends] 
# make a dictionary that maps friends to their locations 
friend_x_y = Dict(zip(friends,x_y_coord)) 
# make list of requested numbers of dice 
req = [5,7,16,4,11,15,8,10,1,6,18,7] 
# make list of numbers of dice each frient currently has 
has = [15,6,10,1,21,9,9,3,10,5,11,8] 
# make dictionaries mapping friends to their requested dice and current dice 
wants_dice = Dict(zip(friends,req)) 
has_dice = Dict(zip(friends,has));

using JuMP, Clp

m = Model(solver=ClpSolver())

dist = zeros(12,12)
for i =1:12
    for j =1:12
        dist[i,j] = 1.6*sqrt((x_coord[i]-x_coord[j])^2 + (y_coord[i]-y_coord[j])^2)
    end
end

@variable(m, x[friends,friends] >=0)

@objective(m, Min, 0.4*sum(dist[i,j]*x[i,j] for i=1:12, j =1:12))

@constraint(m, a[i in friends], sum(x[i,k] for k in friends) == has[i])
@constraint(m, b[i in friends], sum(x[k,i] for k in friends) == req[i])

#@constraint(m, c[i in friends], sum(x[k,i] for k in friends) + has_dice[i] == wants_dice[i] + sum(x[i,k] for k in friends))

solve(m)

println("Cost ",getobjectivevalue(m))




Cost 153.49484179477358


# Question

In [26]:
The last problem has a problem!
we don't need to drive several times to deliver the dices, therefore the flow matrix should be a symbol matrix.
which means any positive flow should be 1,otherwise 0.
And then we can multiply with the distance to calculate the cost of gas.

But, I don't know how to do that.

LoadError: [91msyntax: extra token "last" after end of expression[39m